In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

#### Plot nuclei positions and velocity

In [2]:
daystring = "5"
spotstring = "12"

In [3]:
show_quiver = True
show_track = False

fpath = "YuLong_Data/day_" + daystring + "/" + spotstring + "spot.csv"

df = pd.read_csv(fpath)
max_frame = np.max(list(df['FRAME']))
max_xlim = np.ceil(np.max(df['POSITION_X'])) + 1
max_ylim = np.ceil(np.max(df['POSITION_Y'])) + 1
max_zlim = np.ceil(np.max(df['POSITION_Z'])) + 1
min_xlim = np.floor(np.min(df['POSITION_X'])) - 1
min_ylim = np.floor(np.min(df['POSITION_Y'])) - 1
min_zlim = np.floor(np.min(df['POSITION_Z'])) - 1
max_lim = np.max([max_xlim, max_ylim, max_zlim])
min_lim = np.min([min_xlim, min_ylim, min_zlim])

dirname = "YuLong_Day" + daystring + "_Spot" + spotstring
odirname = dirname + "_plots"
if os.path.isdir(odirname):
    shutil.rmtree(odirname)
    os.mkdir(odirname)
else:
    os.mkdir(odirname)

for tp in range(1, max_frame+1):
        
    ipath = dirname + os.sep + "YuLong_Day" + daystring + "_Spot" + spotstring + "_Frame" + repr(tp) + ".npz"
    track_data = np.load(ipath)
    
    vpath = dirname + os.sep + "YuLong_Day" + daystring + "_Spot" + spotstring + "_Velocity" + repr(tp) + ".npz"
    vel_data = np.load(vpath)
    
    cell_pos = track_data["positions"]
    omega = vel_data["orientation"]
    
    plt.figure(figsize=(5,4), dpi=200)
    ax = plt.axes(projection='3d')
    
    ax.scatter3D(cell_pos[:,0], cell_pos[:,1], cell_pos[:,2], s=4, color='black')
    
    x_com = np.mean(cell_pos[:,0])
    y_com = np.mean(cell_pos[:,1])
    z_com = np.mean(cell_pos[:,2])
    ax.quiver(x_com, y_com, z_com, omega[0], omega[1], omega[2], linewidth=1)
    
    for key in track_data.keys():
        
        if key == "positions":
            continue
        if key == "lumen":
            continue
            
        cell_track = track_data[key]
        x_pos = cell_track[:,0][::-1]
        y_pos = cell_track[:,1][::-1]
        z_pos = cell_track[:,2][::-1]
        
        if show_track:
            ax.plot3D(x_pos, y_pos, z_pos, 'gray', linewidth=0.5)
        
        if show_quiver:
            try:
                cell_vel = vel_data[key]
                ax.quiver(x_pos[-1], y_pos[-1], z_pos[-1], cell_vel[0], cell_vel[1], cell_vel[2], 
                          length=2, normalize=True, linewidth=0.5, color='blue')
            except:
                continue
                
        ax.text(x_pos[-1], y_pos[-1], z_pos[-1], key, color='red', fontsize=4)
        
    outfname = "Frame" + repr(tp).zfill(3) + ".png"
    outfullpath = odirname + os.sep + outfname
    ax.set_xlim([min_lim, max_lim])
    ax.set_ylim([min_lim, max_lim])
    ax.set_zlim([min_lim, max_lim])
    plt.title("Spot " + spotstring)
    plt.savefig(outfullpath)
    plt.close()

In [4]:
! cp make_png_movie.sh YuLong_Day5_Spot12_plots/

#### Plot representative void

In [5]:
dirname = "YuLong_Day" + daystring + "_Spot" + spotstring
odirname = dirname + "_lumen"
if os.path.isdir(odirname):
    shutil.rmtree(odirname)
    os.mkdir(odirname)
else:
    os.mkdir(odirname)

for tp in range(1, max_frame+1):
        
    ipath = dirname + os.sep + "YuLong_Day" + daystring + "_Spot" + spotstring + "_Frame" + repr(tp) + ".npz"
    track_data = np.load(ipath)
    
    cell_pos = track_data["positions"]
    
    lumen_exists = True
    try:
        
        lumen_pts = track_data["lumen"]
        num_triangles = np.shape(lumen_pts)[1]
        tri_data = np.zeros((num_triangles, 9))
    
        for tri in range(num_triangles):

            idx1 = int(lumen_pts[0, tri])-1
            idx2 = int(lumen_pts[1, tri])-1
            idx3 = int(lumen_pts[2, tri])-1

            coords = []
            coords.extend(cell_pos[idx1])
            coords.extend(cell_pos[idx2])
            coords.extend(cell_pos[idx3])
            tri_data[tri,:] = coords
    
        poly3d = [[tri_data[i, j*3:j*3+3] for j in range(3)] for i in range(num_triangles)]
        fc = ["blue" for i in range(num_triangles)]
        ec = ["darkblue" for i in range(num_triangles)]
        
    except:
        lumen_exists = False
    
    fig = plt.figure(figsize=(5,4), dpi=200)
    ax = fig.add_subplot(111, projection='3d')
    
    ax.scatter3D(cell_pos[:,0], cell_pos[:,1], cell_pos[:,2], s=4, color='black')
    
    if lumen_exists:
        ax.add_collection3d(Poly3DCollection(poly3d, facecolors=fc, edgecolors=ec, 
                                             linewidths=0.3, linestyles=':', alpha=0.1))
    
    ax.set_xlim([min_lim, max_lim])
    ax.set_ylim([min_lim, max_lim])
    ax.set_zlim([min_lim, max_lim])
    
    ax.grid(False)
    #plt.axis('off')
    
    outfname = "Frame" + repr(tp).zfill(3) + ".png"
    outfullpath = odirname + os.sep + outfname
    plt.title("Spot " + spotstring)
    plt.savefig(outfullpath)
    plt.close()

In [6]:
! cp make_png_movie.sh YuLong_Day5_Spot12_lumen/